In [5]:
# PATH_NAME = 'C:/Users/gkstk/OneDrive/Desktop/SangMin/Github/AI/data/'
# PATH_NAME2 = 'C:/Users/gkstk/OneDrive/Desktop/SangMin/Github/AI/'


In [6]:
PATH_NAME = 'C:/Users/woobi/Documents/habit/habit-AI/data/'
PATH_NAME2 = 'C:/Users/woobi/Documents/habit/habit-AI/'

In [7]:
import pandas as pd
import numpy as np
import re
from konlpy.tag import Okt
from gensim.models import KeyedVectors
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict
import pickle
import math
from collections import Counter

In [9]:
model = KeyedVectors.load(PATH_NAME + "한국어_음식모델_한상민_v2.kv", mmap='r')
  
wweia_food_categories = pd.read_csv(PATH_NAME + 'wweia_food_categories_addtl.csv')
wweia_data = pd.read_csv(PATH_NAME + 'wweia_data.csv')
wweia_embeddings = pd.read_csv(PATH_NAME + 'word_embeddings.csv', delimiter = ",")

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/woobi/Documents/habit/habit-AI/data/wweia_synonym_cats.pickle'

In [ ]:
stop_words = ['가', '걍', '것', '고', '과', '는', '도', '들', '등', '때', '로', '를', '뿐', '수', '아니', '않', '없', '에', '에게', '와', '으로', '은', '의', '이', '이다', '있', '자', '잘', '좀', '하다', '한', '조각', '개', '것', '대', '소' ,'단계', '등급', '포함', '미니', '개입']

In [ ]:
def reduce_with_food_words(rough_phrase):
  korean_string = re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", " ", rough_phrase)
  okt = Okt()
  token = okt.morphs(korean_string, stem=True)
  
  return token

In [ ]:
def process_food_log(curr_log, wweia_synonym_cats):
  curr_log['predicted_categories_number'] = 0
  curr_log['predicted_categories_words'] = ""
  curr_log['max_cosim_score'] = 0
  curr_log['most_sim_food'] = ""
  curr_log['reciprocal_rank'] = 0.0
  curr_log['sym_reciprocal_rank'] = 0.0

  for i in range(curr_log.shape[0]):
    food_name = curr_log.loc[i, 'Food Name']
    pre_embedding = reduce_with_food_words(food_name)

    word_embed = np.zeros(shape = (1, len(model["불고기"])))
    if len(pre_embedding) > 0:
      
      num_words = 0
      for word in pre_embedding:
        word = word.lower()
        
        if word in model:
          num_words += 1
          word_embed += model[word]

      if num_words != 0:
        word_embed /= num_words
    
    # print("음식 이름")
    # print(food_name)
    # print("프리임베딩")
    # print(pre_embedding)
    # print(word_embed)
    # print('\n')
    
    similarities = cosine_similarity(word_embed, wweia_embeddings)
    to_keep_args = np.argsort(similarities, axis=1)
    indices = np.flip(to_keep_args, axis = 1)

    most_sim_food_row = wweia_data.iloc[indices[0,0], :]
    highest_cat_num = most_sim_food_row['NO']
    highest_cat_words = wweia_food_categories.loc[wweia_food_categories['NO'] == highest_cat_num, '식품명']
    curr_log.loc[i, 'predicted_categories_number'] = highest_cat_num
    curr_log.loc[i, 'predicted_categories_words'] = highest_cat_words.to_list()[0]
    
  return curr_log

In [ ]:
# Main method
def food_recommandation(input_food_list) :
  df = pd.read_csv(PATH_NAME+ "wweia_food_categories_addtl.csv")
  
  numbers = [item[0] for item in input_food_list]
  input_data = df[df['NO'].isin(numbers)]
  print(input_data)
  print("--------------------------------------------------")
  
  input_list= ["wweia_food_category_code", "Food Name", "wweia_food_category_description"]
  curr_log = pd.DataFrame(input_food_list, columns=input_list)
  
  curr_log = process_food_log(curr_log, wweia_synonym_cats)

  print("true 출력 ")
  first_list = set(curr_log.loc[:,'wweia_food_category_code'].tolist())
  first_select_data = df[df['NO'].isin(first_list)]
  print(first_list)
  
  second_list = set(curr_log.loc[:,'predicted_categories_number'].tolist())
  second_select_data = df[df['NO'].isin(second_list)]
  print("pred 출력 ")
  print(second_list)
  
  print("--------------------------------------------------")
  print(first_select_data)
  print("--------------------------------------------------")
  print(second_select_data)

In [ ]:
import pandas as pd
import random

# CSV 파일 읽기
df = pd.read_csv(PATH_NAME+ "wweia_food_categories_addtl.csv")

# 100개의 랜덤한 번호 선택
random_numbers = random.sample(range(1, 7684), 100)

# 선택된 번호에 해당하는 데이터 가져오기
selected_data = df[df['NO'].isin(random_numbers)]


# 필요한 열만 선택하여 리스트로 변환
food_list = selected_data[['NO', '식품명', '식품대분류']].values.tolist()

# 결과 출력
print(food_list)


[[13, '쑥떡', '곡류 및 서류'], [29, '제주도 빙떡 & 귤간장소스', '곡류 및 서류'], [44, '망고 마카롱', '과자류'], [88, '바삭한 우리 찹쌀 모나카', '과자류'], [143, '로스티드 아몬드 앤 초콜릿', '과자류'], [212, '피넛버터쿠키', '과자류'], [253, '고등어석쇠구이', '구이류'], [276, '그릴 더 쏘&윙(소세지+윙)', '구이류'], [396, '사골만두국', '국 및 탕류'], [538, '저염 겉절이', '김치류'], [589, '돼지감자, 삶은것', '농축산물'], [646, '밤, 단택, 삶은것', '농축산물'], [651, '밤, 구운것', '농축산물'], [722, '메밀묵, 가루', '농축산물'], [755, '멥쌀, 백미, 해들, 생것', '농축산물'], [806, '멥쌀, 현미, 일품, 생것', '농축산물'], [924, '칼국수면, 생것', '농축산물'], [984, '보리, 겉보리, 큰알보리1호, 조곡, 생것', '농축산물'], [1043, '옥수수, 메옥수수, 청안옥, 말린것', '농축산물'], [1140, '귤, 만감류, 황금향, 생것', '농축산물'], [1192, '머루, 왕머루, 생것, 껍질과 씨 포함', '농축산물'], [1269, '살구, 잼', '농축산물'], [1390, '솔잎, 추출', '농축산물'], [1430, '오리알, 삶은것', '농축산물'], [1454, '설탕, 가루', '농축산물'], [1511, '두부', '농축산물'], [1514, '두부, 연두부', '농축산물'], [1550, '팥, 적색, 아라리, 말린것', '농축산물'], [1774, '돔, 돌돔, 생것', '농축산물'], [1855, '백연, 생것', '농축산물'], [2006, '잉어, 내장, 생것', '농축산물'], [2199, '조개, 키조개, 근육, 생것', '농축산물'], [2428, '닭고기, 살코기, 생것', '농축산물'], [2551, '햄, 슬라이스햄, 데친것', '

In [ ]:
two_dimensional_array = [
    [1, "메밀전병", "곡류 및 서류"],
    [2, "약식", "곡류 및 서류"],
    [3, "무지개떡", "곡류 및 서류"],
    [45, "초코 마카롱", "과자류"],
    [106, "레몬머랭", "과자류"],
    [105, "코코넛머랭", "과자류"],
    [335, "홍합미역국", "국 및 탕류"]
]
food_recommandation(food_list)

        NO             식품명    식품대분류  식품상세분류
12      13              쑥떡  곡류 및 서류      떡류
28      29  제주도 빙떡 & 귤간장소스  곡류 및 서류      떡류
43      44          망고 마카롱      과자류  기타 과자류
87      88   바삭한 우리 찹쌀 모나카      과자류  기타 과자류
142    143  로스티드 아몬드 앤 초콜릿      과자류    초콜릿류
...    ...             ...      ...     ...
7414  7415      허브치킨 가든샐러드  생채및 무침류     샐러드
7464  7465          양배추샐러드  생채및 무침류     샐러드
7493  7494           북어채무침  생채및 무침류   어패류무침
7537  7538            밤송이게      수산물     갑각류
7667  7668         살오징어-암컷      수산물     두족류

[100 rows x 4 columns]
--------------------------------------------------
true 출력 
{4096, 4097, 2570, 13, 1550, 1043, 4117, 4121, 538, 29, 6690, 44, 6700, 4153, 3642, 3133, 6724, 589, 6737, 88, 3674, 4709, 6245, 3689, 7283, 1140, 7297, 646, 4233, 651, 143, 4241, 5266, 6291, 2199, 6297, 6817, 1192, 5296, 3764, 4789, 6854, 722, 4819, 212, 4314, 6878, 3811, 1774, 755, 1269, 7415, 253, 6929, 276, 6422, 4385, 806, 7465, 5419, 3888, 6962, 1855, 2880, 7494, 2895, 5971, 3924